#  Corn Diseases Detection - Edge Models Training

**Entrenamiento y comparación de 4 arquitecturas edge optimizadas para detección de enfermedades en maíz.**

Este notebook entrena y compara:
- MobileNetV3Large
- EfficientNetLiteB2
- MobileViT (con fine-tuning)
- PMVT (con fine-tuning)

---

##  Configuración Inicial

**Prerequisitos:** 
1. Runtime → Change runtime type → GPU (T4)
2. Sube tu carpeta data/ a Google Drive: MyDrive/corn-diseases-data/
3. Ejecuta todas las celdas en orden

## 1️⃣ Verificar GPU y Recursos

In [ ]:
# Verificar GPU y recursos disponibles
!nvidia-smi
!free -h

import tensorflow as tf
print(f"\n✓ TensorFlow: {tf.__version__}")
print(f"✓ GPU: {len(tf.config.list_physical_devices('GPU'))} disponible(s)")

## 2️⃣ Clonar Repositorio

In [ ]:
!git clone https://github.com/ojgonzalezz/corn-diseases-detection.git
%cd corn-diseases-detection
!ls -la

## 3️⃣ Instalar Dependencias

In [ ]:
!pip install -q -r requirements.txt

# Verificar instalación
import tensorflow as tf
import keras
import mlflow

print("✓ Dependencias instaladas correctamente")
print(f"  TensorFlow: {tf.__version__}")
print(f"  Keras: {keras.__version__}")
print(f"  MLflow: {mlflow.__version__}")

## 4️⃣ Configurar GPU

In [ ]:
import os
import tensorflow as tf

# Variables de entorno
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

# Habilitar memory growth
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    print(f"✓ GPU memory growth habilitado ({len(gpus)} GPU)")

## 5️⃣ Montar Drive y Copiar Datos

In [ ]:
from google.colab import drive
import shutil
import os

# Montar Drive
drive.mount('/content/drive')
print("✓ Google Drive montado")

# Verificar y copiar datos
drive_data_path = '/content/drive/MyDrive/corn-diseases-data'

for split in ['train', 'val', 'test']:
    src = f'{drive_data_path}/{split}'
    dst = f'data/{split}'
    
    if os.path.exists(src):
        if os.path.exists(dst):
            shutil.rmtree(dst)
        shutil.copytree(src, dst)
        print(f"✓ Copiado: {split}")
    else:
        print(f"⚠️ No encontrado: {src}")

## 6️⃣ Verificar Distribución de Datos

In [ ]:
import os

def count_images(path):
    """Cuenta imágenes por clase."""
    counts = {}
    for class_name in os.listdir(path):
        class_path = os.path.join(path, class_name)
        if os.path.isdir(class_path):
            counts[class_name] = len([f for f in os.listdir(class_path) 
                                      if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
    return counts

print("📊 Distribución de datos:\n")
for split in ['train', 'val', 'test']:
    path = f'data/{split}'
    if os.path.exists(path):
        counts = count_images(path)
        total = sum(counts.values())
        print(f"{split.upper()}: {total} imágenes")
        for class_name, count in sorted(counts.items()):
            print(f"  • {class_name}: {count}")
        print()

## 7️⃣ Entrenar Modelos Edge

**Duración estimada: 2-3 horas con GPU T4**

In [ ]:
!python experiments/edge_models/train_all_models.py

## 8️⃣ Comparar Resultados

In [ ]:
!python experiments/edge_models/compare_models.py

## 9️⃣ Seleccionar Mejor Modelo

In [ ]:
!python experiments/edge_models/select_best_model.py

## 🔟 Ver Resultados

In [ ]:
import json
import pandas as pd

# Cargar resultados
with open('experiments/edge_models/best_edge_model.json', 'r') as f:
    best_model = json.load(f)

# Mostrar mejor modelo
print("🏆 MEJOR MODELO SELECCIONADO\n")
print(f"Modelo: {best_model['selected_model']['name']}")
print(f"\n📈 Métricas:")
print(f"  Accuracy: {best_model['performance_metrics']['test_accuracy']:.2%}")
print(f"  Min Recall: {best_model['performance_metrics']['min_recall']:.2%}")
print(f"\n⚙️ Características:")
print(f"  Tamaño: {best_model['model_characteristics']['model_size_mb']} MB")
print(f"  Parámetros: {best_model['model_characteristics']['total_parameters']:,}")
print(f"  Edge-ready: {'✓' if best_model['model_characteristics']['suitable_for_edge'] else '✗'}")

# Comparación completa
df = pd.DataFrame(best_model['all_models_comparison'])
print("\n📊 COMPARACIÓN COMPLETA\n")
print(df.to_string(index=False))

## 1️⃣1️⃣ Descargar Resultados

In [ ]:
# Comprimir resultados
!zip -r edge_models_results.zip \
    experiments/edge_models/best_edge_model.json \
    experiments/edge_models/comparison_results.csv \
    models/exported/*.keras \
    models/exported/*.json \
    models/mlruns/

print("✓ Resultados comprimidos: edge_models_results.zip")

# Copiar a Drive
!cp edge_models_results.zip /content/drive/MyDrive/
print("✓ Guardado en Google Drive")

print("\n📥 Para descargar:")
print("  1. Click en Files (panel izquierdo)")
print("  2. Busca 'edge_models_results.zip'")
print("  3. Click derecho → Download")

## 1️⃣2️⃣ MLFlow UI

In [ ]:
# Visualizar experimentos localmente
print("📊 Para ver experimentos en MLflow:")
print("\n1. Descarga los resultados")
print("2. En tu máquina local ejecuta:")
print("   mlflow ui --backend-store-uri file:///ruta/a/mlruns")
print("3. Abre http://localhost:5000")